In [1]:
"""
windows_setup.py
In order for the Thorlabs Python examples to work, they need visibility of the directory containing the Thorlabs TSI
Native DLLs. This setup function changes the PATH environment variable (Just for the current process, not the system
PATH variable) by adding the directory containing the DLLs. This function is written specifically to work for the
Thorlabs Python SDK examples on Windows, but can be adjusted to work with custom programs. Changing the PATH variable
of a running application is just one way of making the DLLs visible to the program. The following methods could
be used instead:
- Use the os module to adjust the program's current directory to be the directory containing the DLLs.
- Manually copy the DLLs into the working directory of your application.
- Manually add the path to the directory containing the DLLs to the system PATH environment variable.
"""

import os
import sys


def configure_path():
    is_64bits = sys.maxsize > 2**32
    relative_path_to_dlls = '.' + os.sep + 'dlls' + os.sep

    if is_64bits:
        relative_path_to_dlls += '64_lib'
    else:
        relative_path_to_dlls += '32_lib'

    absolute_path_to_file_directory = os.path.dirname(os.path.abspath(__file__))

    absolute_path_to_dlls = os.path.abspath(absolute_path_to_file_directory + os.sep + relative_path_to_dlls)

    os.environ['PATH'] = absolute_path_to_dlls + os.pathsep + os.environ['PATH']

    try:
        # Python 3.8 introduces a new method to specify dll directory
        os.add_dll_directory(absolute_path_to_dlls)
    except AttributeError:
        pass

In [2]:
import ctypes

# Full path to the 'thorlabs_tsi_camera_sdk.dll' file
dll_path = r'C:\Users\athar\OneDrive\Desktop\Unrelated\cam\thorlabs_tsi_camera_sdk.dll'

try:
    # Load the DLL with full path
    my_dll = ctypes.WinDLL(dll_path)
    print("DLL loaded successfully!")
    
    # Now you can access functions from the DLL
    # For example:
    # my_dll.my_function()
    
except Exception as e:
    print("Failed to load DLL:", e)


DLL loaded successfully!


In [4]:
import numpy as np
import os
import cv2
from thorlabs_tsi_sdk.tl_camera import TLCameraSDK, OPERATION_MODE

NUM_FRAMES = 10  # adjust to the desired number of frames
try:
    # if on Windows, use the provided setup script to add the DLLs folder to the PATH
    from windows_setup import configure_path
    configure_path()
except ImportError:
    configure_path = None

with TLCameraSDK() as sdk:
    available_cameras = sdk.discover_available_cameras()
    if len(available_cameras) < 1:
        print("no cameras detected")
    with sdk.open_camera(available_cameras[0]) as camera:
        camera.exposure_time_us = 10000  # set exposure to 11 ms
        camera.frames_per_trigger_zero_for_unlimited = 0  # start camera in continuous mode
        camera.image_poll_timeout_ms = 1000  # 1 second polling timeout
        camera.arm(2)
        camera.issue_software_trigger()
        frame = camera.get_pending_frame_or_null()
        if frame is not None:
            print("frame #{} received!".format(frame.frame_count))
            frame.image_buffer
            image_buffer_copy = np.copy(frame.image_buffer)
            numpy_shaped_image = image_buffer_copy.reshape(camera.image_height_pixels, camera.image_width_pixels)
            nd_image_array = np.full((camera.image_height_pixels, camera.image_width_pixels, 3), 0, dtype=np.uint8)
            nd_image_array[:,:,0] = numpy_shaped_image
            nd_image_array[:,:,1] = numpy_shaped_image
            nd_image_array[:,:,2] = numpy_shaped_image
            
            cv2.imshow("Image From TSI Cam", nd_image_array)
        else:
            print("Unable to acquire image, program exiting...")
            exit()
            
        cv2.waitKey(0)
        camera.disarm()
#  Because we are using the 'with' statement context-manager, disposal has been taken care of.
print("program completed")

frame #1 received!


In [4]:
"""
Polling Example
This example shows how to open a camera, adjust some settings, and poll for images. It also shows how 'with' statements
can be used to automatically clean up camera and SDK resources.
"""
import numpy as np
import os
import cv2
import time
from thorlabs_tsi_sdk.tl_camera import TLCameraSDK, OPERATION_MODE

NUM_FRAMES = 10  # adjust to the desired number of frames
try:
    # if on Windows, use the provided setup script to add the DLLs folder to the PATH
    from windows_setup import configure_path
    configure_path()
except ImportError:
    configure_path = None

with TLCameraSDK() as sdk:
    available_cameras = sdk.discover_available_cameras()
    if len(available_cameras) < 1:
        print("no cameras detected")
    with sdk.open_camera(available_cameras[0]) as camera:
        camera.exposure_time_us = 10000  # set exposure to 11 ms
        camera.frames_per_trigger_zero_for_unlimited = 0  # start camera in continuous mode
        camera.image_poll_timeout_ms = 1000  # 1 second polling timeout
        camera.frame_rate_control_value = 10
        camera.is_frame_rate_control_enabled = True
        camera.arm(2)
        camera.issue_software_trigger()
        try:
            while True:
                frame = camera.get_pending_frame_or_null()
                if frame is not None:
                    print("frame #{} received!".format(frame.frame_count))
                    frame.image_buffer
                    image_buffer_copy = np.copy(frame.image_buffer)
                    numpy_shaped_image = image_buffer_copy.reshape(camera.image_height_pixels, camera.image_width_pixels)
                    nd_image_array = np.full((camera.image_height_pixels, camera.image_width_pixels, 3), 0, dtype=np.uint8)
                    nd_image_array[:,:,0] = numpy_shaped_image
                    nd_image_array[:,:,1] = numpy_shaped_image
                    nd_image_array[:,:,2] = numpy_shaped_image
          
                    cv2.imshow("Image From TSI Cam", nd_image_array)
                    cv2.waitKey(1)
                else:
                    print("Unable to acquire image, program exiting...")
                    exit()
        except KeyboardInterrupt:
            print("loop terminated")
            
        cv2.destroyAllWindows()
        camera.disarm()
#  Because we are using the 'with' statement context-manager, disposal has been taken care of.
print("program completed")

frame #1 received!
frame #3 received!
frame #4 received!
frame #5 received!
frame #6 received!
frame #7 received!
frame #8 received!
frame #9 received!
frame #10 received!
frame #11 received!
frame #12 received!
frame #13 received!
frame #14 received!
frame #15 received!
frame #16 received!
frame #17 received!
frame #18 received!
frame #19 received!
frame #20 received!
frame #21 received!
frame #22 received!
frame #23 received!
frame #24 received!
frame #25 received!
frame #26 received!
frame #27 received!
frame #28 received!
frame #29 received!
frame #30 received!
frame #31 received!
frame #32 received!
frame #33 received!
frame #34 received!
frame #35 received!
frame #36 received!
frame #37 received!
frame #38 received!
frame #39 received!
frame #40 received!
frame #41 received!
frame #42 received!
frame #43 received!
frame #44 received!
frame #45 received!
frame #46 received!
frame #47 received!
frame #48 received!
frame #49 received!
frame #50 received!
frame #51 received!
frame #5